<a href="https://colab.research.google.com/github/sabumjung/Pytorch/blob/master/Autoencoder_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

from keras.datasets import mnist
import numpy as np

# train용과 test용으로 데이터를 가져오기
(x_train, _), (x_test, _) = mnist.load_data()

# train용 데이터에 대한 정규화
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# reshape실행 - flattening
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# train : 60000개, test : 10000개
print(x_train.shape)
print(x_test.shape)

In [0]:
# Deep Autoencoder모델구조 설정
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)


# Model - Optimize - Fitting
autoencoder1 = Model(input_img, decoded)
autoencoder1.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder1.fit(x_train, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test)
)

In [0]:
from keras import regularizers

encoding_dim = 32
input_img = Input(shape=(784,))
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)


# Model - Optimize - Fitting
autoencoder2 = Model(input_img, decoded)
autoencoder2.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder2.fit(x_train, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test)
)

In [0]:
from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format



from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder3 = Model(input_img, decoded)
autoencoder3.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder3.fit(x_train, x_train,
                epochs=20,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

In [0]:
encoded_imgs1 = autoencoder1.predict(x_test)
encoded_imgs2 = autoencoder2.predict(x_test)
encoded_imgs3 = autoencoder3.predict(x_test)
print(encoded_imgs3.mean())

In [0]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [0]:
encoded_imgs = autoencoder.predict(x_test)
print(encoded_imgs.mean())

# use Matplotlib (don't ask)
import matplotlib.pyplot as plt
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(encoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [0]:
import pandas as pd
encoded_imgs = autoencoder.predict(x_test)
print(encoded_imgs.mean())
rmse = pow(np.mean(np.power(x_test - encoded_imgs, 2), axis=1), 0.5)

In [0]:
import matplotlib.pyplot as plt
import numpy as np

o1=encoded_imgs[0].reshape(28*28,1)
o2=x_test[0].reshape(28*28,1)
fig, ax = plt.subplots(1, 1, figsize=(20, 8))
ax.plot(np.arange(28*28), o1, 'b.-', markersize=10)
ax.plot(np.arange(28*28), o2, 'y.-', markersize=10)
ax.grid()
plt.show()
